In [1]:
import numpy as np
import pandas as pd
import gensim
import os
import csv
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path = r"/content/gdrive/My Drive/ANLP-2/train.csv"

with open(path, 'r') as file:
    train_dataset = csv.reader(file)
    next(train_dataset)
    columns = ['value', 'Sentences']
    df = pd.DataFrame(train_dataset, columns=columns)

# df = df[1:]

In [4]:
df['Sentences'][0]

"Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

In [5]:
import spacy, re
from bs4 import BeautifulSoup

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # text = BeautifulSoup(text, "html.parser")
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\\', ' ', text)  # Remove '\\'
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters except whitespace

    return text


def clean_text(text):
    text = preprocess_text(text)
    doc = nlp(text)
    # tokens = [token.lemma_.strip() for token in doc if not token.is_punct and not token.is_stop and token.lemma_.strip() != '']
    tokens = [token.lemma_.strip() for token in doc if not token.is_punct and token.lemma_.strip() != '']
    return tokens

df["Sentences"] = df["Sentences"].str.lower()


In [6]:
html_content = "<p>This is <strong>HTML</strong> content.</p>"

print(clean_text(html_content))

['this', 'be', 'html', 'content']


In [7]:
df.shape

(120000, 2)

In [8]:
# import string,time
# import re
# def remove_tags(text):
#     return text.strip().split()

# punctuations = string.punctuation
# punctuations += "“”•™-_0123456789—"
# def remove_punctuation(text):

#     for char in punctuations:
#         text = text.replace(char,'')
#     return text

# df["Sentences"] = df["Sentences"].str.lower()
# df['Sentences'] = df['Sentences'].apply(remove_punctuation)
# df['Sentences'] = df['Sentences'].apply(remove_tags)


In [9]:
# df["Sentences"] = df["Sentences"].apply(clean_text)

In [10]:
length = len(df)
l1, l2 = int(length*0.7), int(length*0.2)
train, test, dev = df[:l1], df[l1:l1+l2], df[l1+l2:]
print(len(train), len(test), len(dev))

dev = dev.reset_index(drop=True)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)



84000 24000 12000


In [11]:
dev['Sentences'][0]

"the imf has said it will negotiate with argentina after  the country's debt swap, that was delayed until 17 january."

In [12]:
# train['Sentences'] = train['Sentences'].apply(clean_text)

# train_sentence = train['Sentences']
# train_value = train['value']

# train_sentence[0]

In [13]:
# file_path = r"/content/gdrive/My Drive/train_sentence.txt"

# with open(file_path, "w") as file:
#     for sublist in train_sentence:
#         file.write(" ".join(sublist) + "\n")

# file_path = r"/content/gdrive/My Drive/train_value.txt"

# with open(file_path, "w") as file:
#     for item in train_value:
#         file.write(str(item) + "\n")

In [14]:
file_path = r"/content/gdrive/My Drive/train_sentence.txt"

lines = []
with open(file_path, "r") as file:
    lines = file.readlines()
train_sentence = []
for line in lines:
    train_sentence.append(line.split())
print(train_sentence[:2])


file_path = r"/content/gdrive/My Drive/train_value.txt"
lines = []
with open(file_path, "r") as file:
    lines = file.readlines()
train_value = []
for line in lines:
    train_value.append(int(line))

[['reuters', 'shortseller', 'wall', 'streets', 'dwindle', 'band', 'of', 'ultracynic', 'be', 'see', 'green', 'again'], ['reuter', 'private', 'investment', 'firm', 'carlyle', 'group', 'which', 'have', 'a', 'reputation', 'for', 'make', 'welltimed', 'and', 'occasionally', 'controversial', 'play', 'in', 'the', 'defense', 'industry', 'have', 'quietly', 'place', 'its', 'bet', 'on', 'another', 'part', 'of', 'the', 'market']]


In [15]:
import gensim.downloader as api
import torch
import torch.nn as nn

from gensim.models import KeyedVectors
import gensim.downloader as download
glove_model = download.load("glove-wiki-gigaword-100")

glove_embedding_dim = 100

[==================================================] 100.0% 128.1/128.1MB downloaded


In [16]:
word2idx = {'<PAD>': 0, '<UNK>': 1, '<EOS>' : 2, '<SOS>' : 3}
idx2word = {0 : '<PAD>', 1:'<UNK>', 2 : '<EOS>', 3 : '<SOS>'}
idx = 4
vocabulary = set()
vocabulary.add('<PAD>')
vocabulary.add('<UNK>')
vocabulary.add('<SOS>')
vocabulary.add('<EOS>')
for sent in train_sentence:
    for word in sent:
        vocabulary.add(word)

idx = 0
for word in vocabulary:
    word2idx[word] = idx
    idx2word[idx] = word
    idx += 1

In [17]:
word2idx['<PAD>']

29544

In [18]:
print(len(vocabulary), vocabulary)

53639 {'smobileandwirelesstechnology', 'prominence', 'quotmsn', 'tunesmith', 'kapro', 'dinar', 'yeserday', 'dlj', 'targetstocksquickinfofullquotegtusbnltagt', 'antirealnetwork', 'overpeer', 'combo', 'intothebreach', 'mw', 'singleprocessor', 'hrefhttpwwwinvestorreuterscomfullquoteaspxtickercmen', 'seattle', 'gardez', 'episode', 'secularity', 'penultimate', 'copernic', 'southwark', 'michbased', 'fart', 'subang', 'hawkish', 'yorklondon', 'pendleton', 'hostage', 'vue', 'processore', 'iberia', 'holyoke', 'cardstmobile', 'drafting', 'etag', 'unnerve', 'kildall', 'tomorow', 'mikulski', 'homeheating', 'timer', 'webberltpgtltpgt', 'select', 'propel', 'crsb', 'comes', 'expressnews', 'jumpoff', 'counsel', 'shrenepoa', 'qcatnewsgtertsoltagt', 'windowtolinux', 'sequences', 'billpaye', 'brevity', 'hennessy', 'lowsalt', 'cuisine', 'janitorial', 'amos', 'dranke', 'dimunitive', 'linuxinsider', 'cornwell', 'systemic', 'ultracynic', 'straighten', 'filipina', 'mcarthur', 'quotvassell', 'imi', 'fec', 'dodg

In [19]:
unk_vector = np.ones(100)
pad_vector = np.zeros(100)
sos_vector = np.random.rand(100)
eos_vector = np.random.rand(100)

embedding_vector = {'<PAD>':unk_vector, '<SOS>':sos_vector, '<UNK>':unk_vector, '<EOS>':eos_vector}

def get_word_embedding(word):
      if word in glove_model:
          return glove_model[word]
      elif word in embedding_vector:
          return embedding_vector[word]
      else:
          return unk_vector


In [20]:
embedding_matrix = []
for word in vocabulary:
    embedding_matrix.append(get_word_embedding(word))

embedding_matrix = torch.tensor(embedding_matrix)


<ipython-input-20-2f4ec6e2c508>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  embedding_matrix = torch.tensor(embedding_matrix)


In [21]:
# embedding_matrix = np.array(embedding_matrix)

In [22]:
import torch

def custom_collate_fn(sentences):
    # print(sentences)
    max_length = max(len(sentence) for sentence in sentences)
    padded_inputs = []
    padded_outputs = []
    for sentence in sentences:

        inputs, outputs = sentence[:-1], sentence
        # print(inputs)
        # print(outputs)
        # print('='*50)
        # padded_sentence = [word2idx['<SOS>']] + [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in sentence] + [word2idx['<PAD>'] * (max_length - len(sentence))] + [word2idx['<EOS>']]
        sos_token = [word2idx['<SOS>']]
        sentence_tokens = [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in inputs]
        pad_tokens = [word2idx['<PAD>']] * (max_length - len(inputs) -1)
        # eos_token = [word2idx['<EOS>']]
        combined_list = sos_token + sentence_tokens + pad_tokens
        padded_inputs.append(combined_list)

        # sos_token = [word2idx['<SOS>']]
        sentence_tokens = [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in outputs]
        pad_tokens = [word2idx['<PAD>']] * (max_length - len(outputs))
        eos_token = [word2idx['<EOS>']]
        # combined_list = sentence_tokens + pad_tokens + eos_token
        combined_list = sentence_tokens + pad_tokens
        padded_outputs.append(combined_list)

    # print(torch.tensor(padded_inputs).shape, torch.tensor(padded_outputs).shape)
    return torch.tensor(padded_inputs), torch.tensor(padded_outputs)


In [23]:
inp, out = custom_collate_fn(train_sentence[:2])
# inp, out

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

class CustomSentenceDataset(Dataset):
    def __init__(self, list_of_sentences):
        self.list_of_sentences = list_of_sentences

    def __len__(self):
        return len(self.list_of_sentences)

    def __getitem__(self, idx):
        return self.list_of_sentences[idx]

def custom_collate_fn(batch):
    max_len = max(len(sentence) for sentence in batch)
    padded_sentences = [torch.tensor(sentence + [0] * (max_len - len(sentence)) if len(sentence) < max_len else sentence) for sentence in batch]
    return pad_sequence(padded_sentences, batch_first=True)

if __name__ == '__main__':
    # Five lists of sentences with different lengths
    list_of_sentences = [
        [1, 2, 3, 4],
        [5, 6],
        [7, 8, 9, 10, 11],
        [12],
        [13, 14, 15]
    ]

    custom_dataset = CustomSentenceDataset(list_of_sentences)

    # Create a DataLoader with the custom collate function
    batch_size = 2
    dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)

    for batch in dataloader:
        print("Batch of padded sentences:\n", batch)


In [24]:
from torch.utils.data import DataLoader
batch_size = 32
dataloader = DataLoader(train_sentence, batch_size=batch_size, collate_fn=custom_collate_fn)


In [25]:
len(train_sentence), len(dataloader), len(train_sentence)/batch_size

(84000, 2625, 2625.0)

In [26]:
idx2word[2], idx2word[5], idx2word[7]

('quotmsn', 'dinar', 'dlj')

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [28]:
# import torch

# index_sentences = torch.tensor([[1, 2, 3, 4], [4, 5, 6, 7]])
# x = index_sentences.view(-1, len(x[0]))
# embeddings = embedding_matrix[x]
# embeddings.shape, embeddings

In [29]:
# import torch
# import torch.nn as nn

# class ELMo(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim=100, num_layers=1, dropout=0.3):
#         super(ELMo, self).__init__()

#         # Word embeddings layer (you can replace this with Word2Vec embeddings)
#         self.hidden_dim = hidden_dim
#         self.vocab_size = vocab_size
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm_forward_1 = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True).double()
#         self.lstm_forward_2 = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True).double()
#         self.lstm_backward_1 = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True).double()
#         self.lstm_backward_2 = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True).double()
#         self.scalars = nn.ParameterList([nn.Parameter(torch.ones(2)) for _ in range(num_layers+1)]).double()
#         # self.linear_layer = nn.Linear(hidden_dim*2, vocab_size).double()  # Set the output dimension to 100
#         self.dropout = nn.Dropout(dropout).double()

#     def forward(self, input):
#         input = input.view(-1, len(input[0]))
#         embeddings = embedding_matrix[input]
#         lstm_outputs = [[embeddings, embeddings]]

#         lstm_out_forward_1, _ = self.lstm_forward_1(embeddings)
#         # embeddings_reversed_1 = torch.flip(embeddings, [1])
#         lstm_out_backward_1, _ = self.lstm_backward_1(torch.flip(embeddings, [1]))
#         lstm_out_backward_1 = torch.flip(lstm_out_backward_1, [1])

#         lstm_out_forward_2, _ = self.lstm_forward_2(lstm_out_forward_1)
#         # embeddings_reversed_2 = torch.flip(lstm_out_backward_1, [1])
#         lstm_out_backward_2, _ = self.lstm_backward_2(torch.flip(lstm_out_backward_1, [1]))
#         lstm_out_backward_2 = torch.flip(lstm_out_backward_2, [1])

        # lstm_outputs.append([lstm_out_forward_1, lstm_out_backward_1])
        # lstm_outputs.append([lstm_out_forward_2, lstm_out_backward_2])

#         combined_output = torch.zeros((batch_size, len(input[0]), 200), dtype=torch.float64).to(device)

        # for i, lstm_out in enumerate(lstm_outputs):
        #     tensor1 = self.scalars[i][0] * lstm_out[0]
        #     tensor2 = self.scalars[i][1] * lstm_out[1]

        #     concatenated = torch.cat((tensor1, tensor2), dim=2)
        #     print(concatenated.shape)
        #     combined_output += concatenated

#         print(combined_output.shape)
#         linear_layer = nn.Linear(self.hidden_dim*2, self.vocab_size*len(input[0])).double().to(device)
#         linear_output = linear_layer(combined_output).view(-1, batch_size).view(-1, len(input[0]))

#         return linear_output


PARAMETERS TRAINABLE

In [ ]:
class ELMo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.scalars = nn.ParameterList([nn.Parameter(torch.ones(1)) for _ in range(num_layers+1)]).double()
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        # self.alphas = [0.2, 0.4, 0.4]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)
        embed_out = embedded_x*self.scalars[0] + lstm_out1*self.scalars[1] + lstm_out2*self.scalars[2]
        return final_out, embed_out


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
embedding_dim = 100
hidden_dim = 50
num_layers = 2
batch_size_lstm = 32
vocab_size = len(vocabulary)
embedding_matrix = embedding_matrix.to(device)
model = ELMo(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-29-fb1335d3a194>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [ ]:
for batch_input_embeddings, batch_target_embeddings in (dataloader):
    print(batch_input_embeddings, batch_target_embeddings)
    print(batch_input_embeddings.shape, batch_target_embeddings.shape)
    break

tensor([[49660, 35702, 40248,  ..., 15807, 15807, 15807],
        [49660, 50622, 38098,  ..., 15807, 15807, 15807],
        [49660, 50622,  2646,  ..., 15807, 15807, 15807],
        ...,
        [49660, 18230, 17335,  ..., 15807, 15807, 15807],
        [49660, 11169, 43458,  ..., 15807, 15807, 15807],
        [49660, 21282,  1731,  ..., 15807, 15807, 15807]]) tensor([[35702, 40248,  1266,  ..., 15807, 15807, 15807],
        [50622, 38098,  4484,  ..., 15807, 15807, 15807],
        [50622,  2646, 39906,  ..., 15807, 15807, 15807],
        ...,
        [18230, 17335, 11169,  ..., 15807, 15807, 15807],
        [11169, 43458, 23687,  ..., 15807, 15807, 15807],
        [21282,  1731, 11169,  ..., 15807, 15807, 15807]])
torch.Size([32, 62]) torch.Size([32, 62])


In [ ]:
num_epochs = 5
path = "/content/gdrive/My Drive/ass2_model_parameters.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        _, outputs = model(torch.tensor(batch_input_embeddings))
        # outputs = outputs.float()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')

  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-33-13f6fb4251a0>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model(torch.tensor(batch_input_embeddings))
 20%|██        | 1/5 [17:51<1:11:27, 1071.75s/it]

Epoch [1/5], Loss: 1.3540230814070338


 40%|████      | 2/5 [35:45<53:38, 1072.72s/it]  

Epoch [2/5], Loss: 0.7099486117042798


 60%|██████    | 3/5 [53:36<35:44, 1072.12s/it]

Epoch [3/5], Loss: 0.648849867084227


 80%|████████  | 4/5 [1:11:28<17:52, 1072.14s/it]

Epoch [4/5], Loss: 0.6242876896055104


100%|██████████| 5/5 [1:29:21<00:00, 1072.36s/it]

Epoch [5/5], Loss: 0.6116380488229803


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class downstream_task(nn.Module):
    def __init__(self, embedding_dim, vocab_size, no_of_values, num_layers=2):
        super(downstream_task, self).__init__()
        self.vocab_size = vocab_size
        self.no_of_values = no_of_values
        # self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers, batch_first=True).double()
        self.linear = nn.Linear(embedding_dim, no_of_values).double()

    def forward(self, input):
        # lstm_out, (hidden, cell_state) = self.lstm(input)
        # print("hidden shape", hidden.shape)
        # linear_out = self.linear(hidden[1])
        # print("linear_out shape", linear_out.shape)

        linear_out = self.linear(input)
        return linear_out

In [ ]:
model_downstream = downstream_task(embedding_dim, vocab_size, 4).to(device)
no_of_values = 4
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_downstream.parameters(), lr=learning_rate, weight_decay=1e-5)
train_val = [int(val)-1 for val in train_value]

In [ ]:
import torch

def custom_collate_fn(batch):
    max_length = max(len(sentence) for sentence, _ in batch)
    padded_inputs = []
    outputs = []
    for sentence, label in batch:
        # print(sentence)
        sos_token = [word2idx['<SOS>']]
        sentence_tokens = [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in sentence]
        pad_tokens = [word2idx['<PAD>']] * (max_length - len(sentence))
        combined_list = sos_token + sentence_tokens + pad_tokens

        padded_inputs.append(combined_list)
        outputs.append(label)


    # print(torch.tensor(padded_inputs).shape, torch.tensor(outputs).shape)
    return torch.tensor(padded_inputs), torch.tensor(outputs)


In [ ]:


for batch_input_embeddings, batch_target in (dataloader_downstream):
    print(batch_input_embeddings, batch_target)
    print(batch_input_embeddings.shape, batch_target.shape)
    break

tensor([[49660, 35702, 40248,  ..., 15807, 15807, 15807],
        [49660, 50622, 38098,  ..., 15807, 15807, 15807],
        [49660, 50622,  2646,  ..., 15807, 15807, 15807],
        ...,
        [49660, 18230, 17335,  ..., 15807, 15807, 15807],
        [49660, 11169, 43458,  ..., 15807, 15807, 15807],
        [49660, 21282,  1731,  ..., 15807, 15807, 15807]]) tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([32, 63]) torch.Size([32])


In [ ]:
num_epochs = 5
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model(batch_input_embeddings)
        # print("outputs",outputs.shape)

        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_parameters_model.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-37-d7810332b40a>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model(torch.tensor(batch_input_embeddings))
 20%|██        | 1/5 [18:33<1:14:13, 1113.26s/it]

Epoch [1/5], Loss: 1.3720245017356794


 40%|████      | 2/5 [37:09<55:45, 1115.13s/it]  

Epoch [2/5], Loss: 0.6849044107602886


 60%|██████    | 3/5 [55:42<37:08, 1114.16s/it]

Epoch [3/5], Loss: 0.6332864628748438


 80%|████████  | 4/5 [1:14:16<18:33, 1113.86s/it]

Epoch [4/5], Loss: 0.6118473768733342


100%|██████████| 5/5 [1:32:50<00:00, 1114.02s/it]

Epoch [5/5], Loss: 0.6002340731082593


In [ ]:
test['Sentences'] = test['Sentences'].apply(clean_text)

test_sentence = test['Sentences']
test_value = test['value']

custom_dataset_test = CustomDataset(test_sentence, test_value)
dataloader_downstream_test = DataLoader(custom_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)

In [ ]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_test):

for batch_input_embeddings, batch_target in dataloader_downstream_test:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model(batch_input_embeddings)
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]

    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()

    # print("Input 1 : ", batch_input_embeddings.shape)
    # print("Output 1 : ", outputs.shape)
    # print(batch_input_embeddings)
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    # outputs_sum = outputs_sum.cpu().numpy()
    # print("argmax : ", outputs_down)

    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()
    # print("Output 2 : ", outputs_downstream.shape)



    true.append(batch_target.cpu())  # Move the true labels to CPU
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    # if(total == 10):
    #     break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.73
0.74
0.72
0.6875
0.694
0.6983333333333334
0.7028571428571428
0.70125
0.7
0.698
0.6954545454545454
0.6975
0.6992307692307692
0.695
0.6953333333333334
0.69875
0.7
0.7
0.7005263157894737
0.6935
0.6952380952380952
0.6963636363636364
0.6921739130434783
0.69125
0.6916
0.6942307692307692
0.6937037037037037
0.6953571428571429
0.696551724137931
0.6943333333333334
0.6948387096774193
0.696875
0.696060606060606
0.6952941176470588
0.6951428571428572
0.6958333333333333
0.6940540540540541
0.695
0.6935897435897436
0.69275
0.6936585365853658
0.6945238095238095
0.6948837209302325
0.6959090909090909
0.6946666666666667
0.6952173913043478
0.6959574468085107
0.695625
0.6961224489795919
0.6972
0.6962745098039216
0.6951923076923077
0.6962264150943396
0.6975925925925925
0.6958181818181818
0.6951785714285714
0.6959649122807018
0.6946551724137932
0.6942372881355933
0.6963333333333334
0.6955737704918032
0.6958064516129032
0.6973015873015873
0.69640625
0.6952307692307692
0.6954545454545454
0.6949253731343283


In [ ]:
print(true)
print(predicted)

true1 = true
true = [int(val) for val in true]

[tensor([1]), tensor([0]), tensor([3]), tensor([2]), tensor([1]), tensor([2]), tensor([3]), tensor([0]), tensor([1]), tensor([2]), tensor([3]), tensor([0]), tensor([2]), tensor([3]), tensor([1]), tensor([2]), tensor([0]), tensor([3]), tensor([2]), tensor([2]), tensor([0]), tensor([0]), tensor([3]), tensor([1]), tensor([2]), tensor([0]), tensor([3]), tensor([1]), tensor([1]), tensor([0]), tensor([3]), tensor([2]), tensor([3]), tensor([0]), tensor([2]), tensor([3]), tensor([2]), tensor([3]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([3]), tensor([0]), tensor([3]), tensor([2]), tensor([0]), tensor([1]), tensor([3]), tensor([3]), tensor([1]), tensor([0]), tensor([1]), tensor([1]), tensor([2]), tensor([2]), tensor([1]), tensor([2]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([1]), tensor([1]), tensor([2]), tensor([2]), tensor([0]), tensor([0]), tensor([2]), tensor([1]), tensor([2]), tensor([3]), tensor([1]), tensor([1]), tensor([2]), tensor([0]), tensor([1])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.6975
Recall: 0.6975
F1 Score: 0.6975
Confusion Matrix:
[[15891  1639  2526  1328]
 [ 2417 17523   319   600]
 [ 3268   605 13856  2895]
 [ 3657  1165  4991 11320]]


ELMO WITH DROPOUT AND PARAMETER TRAINED

In [ ]:
class ELMo_dropout(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo_dropout, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.scalars = nn.ParameterList([nn.Parameter(torch.ones(1)) for _ in range(num_layers+1)]).double()
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=0.3, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=0.3, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        # self.alphas = [0.2, 0.4, 0.4]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)
        embed_out = embedded_x*self.scalars[0] + lstm_out1*self.scalars[1] + lstm_out2*self.scalars[2]
        return final_out, embed_out


In [ ]:
model_dropout = ELMo_dropout(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
# learning_rate = 0.001
optimizer = optim.Adam(model_dropout.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-38-0bd349b711b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
num_epochs = 5
path = "/content/gdrive/My Drive/ass2_model_dropout.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        _, outputs = model_dropout(torch.tensor(batch_input_embeddings))
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-40-2b22d6d85ac0>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model_dropout(torch.tensor(batch_input_embeddings))
 20%|██        | 1/5 [17:51<1:11:24, 1071.05s/it]

Epoch [1/5], Loss: 1.30987418419535


 40%|████      | 2/5 [35:44<53:37, 1072.34s/it]  

Epoch [2/5], Loss: 0.6748076834980786


 60%|██████    | 3/5 [53:36<35:44, 1072.06s/it]

Epoch [3/5], Loss: 0.6295672774012412


 80%|████████  | 4/5 [1:11:27<17:51, 1071.81s/it]

Epoch [4/5], Loss: 0.6113761594601801


100%|██████████| 5/5 [1:29:19<00:00, 1071.84s/it]

Epoch [5/5], Loss: 0.6005370910112979


In [ ]:
# train_val = [int(val)-1 for val in train_value]
model_downstream_dropout = downstream_task(embedding_dim, vocab_size, 4).to(device)
optimizer = optim.Adam(model_downstream_dropout.parameters(), lr=0.01, weight_decay=1e-5)


In [ ]:
num_epochs = 5
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream_test):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model_dropout(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream_dropout(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream_dropout.state_dict(), "/content/gdrive/My Drive/ass2_downstream_dropout_model.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream_test)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-43-8b86c813e530>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
 20%|██        | 1/5 [05:44<22:56, 344.01s/it]

Epoch [1/5], Loss: 0.901450421695351


 40%|████      | 2/5 [11:29<17:15, 345.06s/it]

Epoch [2/5], Loss: 0.8270333463456792


 60%|██████    | 3/5 [17:15<11:30, 345.26s/it]

Epoch [3/5], Loss: 0.812815165615335


 80%|████████  | 4/5 [22:59<05:44, 344.70s/it]

Epoch [4/5], Loss: 0.8036834058426049


100%|██████████| 5/5 [28:43<00:00, 344.64s/it]

Epoch [5/5], Loss: 0.7971221851240726


In [4]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_test):

for batch_input_embeddings, batch_target in dataloader_downstream_test:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model_dropout(batch_input_embeddings)
    outputs = outputs.detach()
    outputs_downstream = model_downstream_dropout(outputs).cpu().numpy()
    outputs_down = np.argmax(F.softmax(torch.tensor(outputs_downstream), dim=1).cpu().numpy())
    true.append(batch_target.cpu())  # Move the true labels to CPU
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    if(total > 1000):
        break

print("accuracy : ", count / total)


accuracy :  0.798214285714285


In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.7982142857142858
Recall: 0.7982142857142858
F1 Score: 0.7982142857142858
Confusion Matrix:
[[386  32  35  26]
 [ 29 387  26  24]
 [ 18  30 259  37]
 [ 22  32  28 309]]


MODEL WITH FIXED PARAMETRES

In [30]:
class ELMo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        self.alphas = [0.2, 0.4, 0.4]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        # print("lstm1 : ", lstm_out1.shape)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)

        embed_out = embedded_x*self.alphas[0] + lstm_out1*self.alphas[1] + lstm_out2*self.alphas[2]
        return final_out, embed_out


In [31]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
embedding_dim = 100
hidden_dim = 50
num_layers = 2
batch_size_lstm = 32
vocab_size = len(vocabulary)
embedding_matrix = embedding_matrix.to(device)
model = ELMo(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-30-54ae2f4af395>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [32]:
idx2word[52237]

'jeered'

In [33]:
for batch_input_embeddings, batch_target_embeddings in (dataloader):
    print(batch_input_embeddings, batch_target_embeddings)
    print(batch_input_embeddings.shape, batch_target_embeddings.shape)
    break

tensor([[ 9437, 19339, 53582,  ..., 29544, 29544, 29544],
        [ 9437,  6148, 32187,  ..., 29544, 29544, 29544],
        [ 9437,  6148, 50337,  ..., 29544, 29544, 29544],
        ...,
        [ 9437, 40348, 44712,  ..., 29544, 29544, 29544],
        [ 9437, 50055, 15189,  ..., 29544, 29544, 29544],
        [ 9437, 49183, 49419,  ..., 29544, 29544, 29544]]) tensor([[19339, 53582, 29201,  ..., 29544, 29544, 29544],
        [ 6148, 32187,  4271,  ..., 29544, 29544, 29544],
        [ 6148, 50337, 14321,  ..., 29544, 29544, 29544],
        ...,
        [40348, 44712, 50055,  ..., 29544, 29544, 29544],
        [50055, 15189, 50830,  ..., 29544, 29544, 29544],
        [49183, 49419, 50055,  ..., 29544, 29544, 29544]])
torch.Size([32, 62]) torch.Size([32, 62])


In [ ]:
num_epochs = 5
path = "/content/gdrive/My Drive/ass2_model.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        _, outputs = model(torch.tensor(batch_input_embeddings))
        outputs = outputs.long()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-35-6d83e213ff55>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model(torch.tensor(batch_input_embeddings))
 20%|██        | 1/5 [17:14<1:08:57, 1034.39s/it]

Epoch [1/5], Loss: 1.3446232639210565


 40%|████      | 2/5 [34:29<51:44, 1034.77s/it]  

Epoch [2/5], Loss: 0.7201183559326898


 60%|██████    | 3/5 [51:43<34:28, 1034.23s/it]

Epoch [3/5], Loss: 0.6588114766336622


 80%|████████  | 4/5 [1:08:56<17:14, 1034.06s/it]

Epoch [4/5], Loss: 0.6326542115381786


100%|██████████| 5/5 [1:26:09<00:00, 1033.97s/it]

Epoch [5/5], Loss: 0.619385635864167


In [37]:
num_epochs = 5
path = "/content/gdrive/My Drive/ass2_model.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        outputs, _ = model(torch.tensor(batch_input_embeddings))
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-37-d7810332b40a>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model(torch.tensor(batch_input_embeddings))
 20%|██        | 1/5 [18:33<1:14:13, 1113.26s/it]

Epoch [1/5], Loss: 1.3720245017356794


 40%|████      | 2/5 [37:09<55:45, 1115.13s/it]  

Epoch [2/5], Loss: 0.6849044107602886


 60%|██████    | 3/5 [55:42<37:08, 1114.16s/it]

Epoch [3/5], Loss: 0.6332864628748438


 80%|████████  | 4/5 [1:14:16<18:33, 1113.86s/it]

Epoch [4/5], Loss: 0.6118473768733342


100%|██████████| 5/5 [1:32:50<00:00, 1114.02s/it]

Epoch [5/5], Loss: 0.6002340731082593


In [2]:
# path = "/content/gdrive/My Drive/ass2_model.pth"
# model = torch.load(path, map_location=torch.device('cpu'))


model.load_state_dict(torch.load(path))

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class downstream_task(nn.Module):
    def __init__(self, embedding_dim, vocab_size, no_of_values, num_layers=2):
        super(downstream_task, self).__init__()
        self.vocab_size = vocab_size
        self.no_of_values = no_of_values
        # self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers, batch_first=True).double()
        self.linear = nn.Linear(embedding_dim, no_of_values).double()

    def forward(self, input):
        # lstm_out, (hidden, cell_state) = self.lstm(input)
        # print("hidden shape", hidden.shape)
        # linear_out = self.linear(hidden[1])
        # print("linear_out shape", linear_out.shape)

        linear_out = self.linear(input)
        return linear_out

In [35]:
model_downstream = downstream_task(embedding_dim, vocab_size, 4).to(device)

# num_params = sum(p.numel() for p in model_downstream.parameters() if p.requires_grad)
# print(f"Number of trainable parameters: {num_params}")

no_of_values = 4

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_downstream.parameters(), lr=learning_rate, weight_decay=1e-5)
train_val = [int(val)-1 for val in train_value]

In [36]:
import torch

def custom_collate_fn(batch):
    max_length = max(len(sentence) for sentence, _ in batch)
    padded_inputs = []
    outputs = []
    for sentence, label in batch:
        # print(sentence)
        sos_token = [word2idx['<SOS>']]
        sentence_tokens = [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in sentence]
        pad_tokens = [word2idx['<PAD>']] * (max_length - len(sentence))
        combined_list = sos_token + sentence_tokens + pad_tokens

        padded_inputs.append(combined_list)
        outputs.append(label)


    # print(torch.tensor(padded_inputs).shape, torch.tensor(outputs).shape)
    return torch.tensor(padded_inputs), torch.tensor(outputs)


In [37]:

from torch.utils.data import DataLoader, Dataset
batch_size = 32

class CustomDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return [self.sentences[idx], self.labels[idx]]

batch_size = 32
custom_dataset = CustomDataset(train_sentence, train_val)
dataloader_downstream = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)



In [38]:
for batch_input_embeddings, batch_target in (dataloader_downstream):
    print(batch_input_embeddings, batch_target)
    print(batch_input_embeddings.shape, batch_target.shape)
    break

tensor([[ 9437, 19339, 53582,  ..., 29544, 29544, 29544],
        [ 9437,  6148, 32187,  ..., 29544, 29544, 29544],
        [ 9437,  6148, 50337,  ..., 29544, 29544, 29544],
        ...,
        [ 9437, 40348, 44712,  ..., 29544, 29544, 29544],
        [ 9437, 50055, 15189,  ..., 29544, 29544, 29544],
        [ 9437, 49183, 49419,  ..., 29544, 29544, 29544]]) tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([32, 63]) torch.Size([32])


In [43]:
num_epochs = 5
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_non_parametered_model.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-43-b5dcfad06de3>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
 20%|██        | 1/5 [05:57<23:48, 357.21s/it]

Epoch [1/5], Loss: 1.2292924365109108


 40%|████      | 2/5 [11:52<17:47, 356.00s/it]

Epoch [2/5], Loss: 1.041343223710708


 60%|██████    | 3/5 [17:47<11:50, 355.45s/it]

Epoch [3/5], Loss: 0.9544469316660587


 80%|████████  | 4/5 [23:41<05:55, 355.14s/it]

Epoch [4/5], Loss: 0.9076114486524991


100%|██████████| 5/5 [29:36<00:00, 355.29s/it]

Epoch [5/5], Loss: 0.8796169408547623


In [ ]:
num_epochs = 5
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_non_parametered_model.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-85-65d794406ea1>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
 20%|██        | 1/5 [05:51<23:25, 351.39s/it]

Epoch [1/5], Loss: 1.0310005471383792


 40%|████      | 2/5 [11:42<17:33, 351.31s/it]

Epoch [2/5], Loss: 0.9844350715939316


 60%|██████    | 3/5 [17:34<11:42, 351.45s/it]

Epoch [3/5], Loss: 0.954049007221359


 80%|████████  | 4/5 [23:25<05:51, 351.35s/it]

Epoch [4/5], Loss: 0.9325959116734661


100%|██████████| 5/5 [29:16<00:00, 351.35s/it]

Epoch [5/5], Loss: 0.9165826937655669


In [ ]:
num_epochs = 5
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model(batch_input_embeddings)
        print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        print("outputs_downstream 1 : ",outputs_downstream.shape)

        outputs_downstream = model_downstream(outputs_sum)
        print("outputs_downstream 2 : ",outputs_downstream.shape)
        print("Want : ", batch_target.shape)
        loss = loss_function(outputs_downstream, batch_target)
        break
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-99-b9668a077957>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)


outputs torch.Size([32, 63, 100])
outputs_downstream 1 :  torch.Size([32, 4])
outputs_downstream 2 :  torch.Size([32, 4])
Want :  torch.Size([32])
Epoch [1/5], Loss: 0.0


 60%|██████    | 3/5 [00:00<00:00,  7.74it/s]

outputs torch.Size([32, 63, 100])
outputs_downstream 1 :  torch.Size([32, 4])
outputs_downstream 2 :  torch.Size([32, 4])
Want :  torch.Size([32])
Epoch [2/5], Loss: 0.0
outputs torch.Size([32, 63, 100])
outputs_downstream 1 :  torch.Size([32, 4])
outputs_downstream 2 :  torch.Size([32, 4])
Want :  torch.Size([32])
Epoch [3/5], Loss: 0.0


100%|██████████| 5/5 [00:00<00:00,  8.01it/s]

outputs torch.Size([32, 63, 100])
outputs_downstream 1 :  torch.Size([32, 4])
outputs_downstream 2 :  torch.Size([32, 4])
Want :  torch.Size([32])
Epoch [4/5], Loss: 0.0
outputs torch.Size([32, 63, 100])
outputs_downstream 1 :  torch.Size([32, 4])
outputs_downstream 2 :  torch.Size([32, 4])
Want :  torch.Size([32])
Epoch [5/5], Loss: 0.0


In [44]:
# dev['Sentences'] = dev['Sentences'].apply(clean_text)

# dev_sentence = dev['Sentences']
# dev_value = dev['value']

In [46]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_test):

for batch_input_embeddings, batch_target in dataloader_downstream_test:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model(batch_input_embeddings)
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    outputs = outputs.detach()
    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()

    # print("Input 1 : ", batch_input_embeddings.shape)
    # print("Output 1 : ", outputs.shape)
    # print(batch_input_embeddings)
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    # outputs_sum = outputs_sum.cpu().numpy()
    # print("argmax : ", outputs_down)

    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()
    # print("Output 2 : ", outputs_downstream.shape)



    true.append(batch_target.cpu())  # Move the true labels to CPU
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    # if(total == 10):
    #     break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.71
0.655
0.6333333333333333
0.6625
0.654
0.6566666666666666
0.6685714285714286
0.67
0.6688888888888889
0.661
0.6681818181818182
0.6616666666666666
0.6615384615384615
0.6592857142857143
0.6586666666666666
0.660625
0.658235294117647
0.6594444444444445
0.6589473684210526
0.662
0.6619047619047619
0.6618181818181819
0.662608695652174
0.6629166666666667
0.6628
0.665
0.662962962962963
0.6610714285714285
0.6613793103448276
0.6613333333333333
0.6612903225806451
0.6625
0.6633333333333333
0.6641176470588235
0.6634285714285715
0.6613888888888889
0.6605405405405406
0.6613157894736842
0.6607692307692308
0.66175
0.6607317073170732
0.6597619047619048
0.6613953488372093
0.6625
0.662
0.6602173913043479
0.6608510638297872
0.6610416666666666
0.6610204081632653
0.661
0.6607843137254902
0.6594230769230769
0.659622641509434
0.6594444444444445
0.6601818181818182
0.6583928571428571
0.6584210526315789
0.6582758620689655
0.6584745762711864
0.658
0.6591803278688525
0.6588709677419354
0.6588888888888889
0.658281

In [47]:
print(true)
print(predicted)

[tensor([1]), tensor([3]), tensor([3]), tensor([3]), tensor([1]), tensor([0]), tensor([2]), tensor([3]), tensor([0]), tensor([1]), tensor([3]), tensor([2]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([1]), tensor([1]), tensor([0]), tensor([1]), tensor([3]), tensor([3]), tensor([0]), tensor([0]), tensor([0]), tensor([2]), tensor([2]), tensor([3]), tensor([1]), tensor([0]), tensor([0]), tensor([3]), tensor([0]), tensor([1]), tensor([0]), tensor([2]), tensor([3]), tensor([0]), tensor([3]), tensor([2]), tensor([0]), tensor([3]), tensor([3]), tensor([2]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([3]), tensor([0]), tensor([0]), tensor([2]), tensor([3]), tensor([3]), tensor([0]), tensor([3]), tensor([1]), tensor([3]), tensor([1]), tensor([1]), tensor([1]), tensor([2]), tensor([0]), tensor([1]), tensor([2]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([2]), tensor([1]), tensor([0]), tensor([2]), tensor([1]), tensor([1]), tensor([0]), tensor([0])

In [50]:
true1 = true

# true

In [51]:
true = [int(val) for val in true]
true[0]

1

In [54]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.66375
Recall: 0.66375
F1 Score: 0.66375
Confusion Matrix:
[[14965  2532  2037  1850]
 [ 2159 18018   119   563]
 [ 5075   775 10878  3896]
 [ 3911  1923  3405 11894]]


In [55]:
class ELMo_last_layer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo_last_layer, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.scalars = nn.ParameterList([nn.Parameter(torch.ones(1)) for _ in range(num_layers+1)]).double()
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.scalars = nn.ParameterList([nn.Parameter(torch.ones(1)) for _ in range(1)]).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        # self.alphas = [0.2, 0.4, 0.4]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)

        embed_out = lstm_out2*self.scalars[0]
        return final_out, embed_out


In [56]:
model_ELMo_last_layer = ELMo_last_layer(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-55-5089cd165ea8>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [58]:
num_epochs = 2
path = "/content/gdrive/My Drive/ass2_model_last_parameters.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        _, outputs = model_ELMo_last_layer(torch.tensor(batch_input_embeddings))
        outputs = outputs.detach()
        # outputs = outputs.float()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model_ELMo_last_layer.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')

  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-58-c40973e28898>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model_ELMo_last_layer(torch.tensor(batch_input_embeddings))
 50%|█████     | 1/2 [18:03<18:03, 1083.81s/it]

Epoch [1/2], Loss: 10.87591713621697


100%|██████████| 2/2 [36:06<00:00, 1083.48s/it]

Epoch [2/2], Loss: 10.87591713621697


In [61]:
model_downstream_last_layer = downstream_task(embedding_dim, vocab_size, 4).to(device)
no_of_values = 4
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_downstream_last_layer.parameters(), lr=learning_rate, weight_decay=1e-5)

In [66]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_dev):

for batch_input_embeddings, batch_target in dataloader_downstream_dev:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model_ELMo_last_layer(batch_input_embeddings)
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]

    outputs_downstream = model_downstream_last_layer(outputs_sum).cpu().detach().numpy()
    outputs_downstream = outputs_downstream.detach()
    # print("Input 1 : ", batch_input_embeddings.shape)
    # print("Output 1 : ", outputs.shape)
    # print(batch_input_embeddings)
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    # outputs_sum = outputs_sum.cpu().numpy()
    # print("argmax : ", outputs_down)

    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()
    # print("Output 2 : ", outputs_downstream.shape)



    true.append(batch_target.cpu())  # Move the true labels to CPU
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    if(total == 10000):
        break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.31
0.295
0.2866666666666667
0.275
0.27
0.275
0.26857142857142857
0.26875
0.2677777777777778
0.268
0.26545454545454544
0.26166666666666666
0.26
0.25785714285714284
0.254
0.253125
0.2535294117647059
0.2511111111111111
0.2505263157894737
0.249
0.25
0.2509090909090909
0.24695652173913044
0.25083333333333335
0.2508
0.2523076923076923
0.25074074074074076
0.2507142857142857
0.2496551724137931
0.25133333333333335
0.25
0.2496875
0.2496969696969697
0.24970588235294117
0.24971428571428572
0.24833333333333332
0.2481081081081081
0.24842105263157896
0.25051282051282053
0.2515
0.2519512195121951
0.2523809523809524
0.25232558139534883
0.25113636363636366
0.24955555555555556
0.24956521739130436
0.24914893617021278
0.24854166666666666
0.24816326530612245
0.2488
0.24941176470588236
0.2501923076923077
0.250377358490566
0.2501851851851852
0.25036363636363634
0.25035714285714283
0.24964912280701754
0.2489655172413793
0.24915254237288137
0.24866666666666667
0.24770491803278688
0.24725806451612903
0.2476190

In [67]:
true = [tensor.item() for tensor in true]

In [68]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.2455
Recall: 0.2455
F1 Score: 0.2455
Confusion Matrix:
[[   1  321 2228    0]
 [   8  154 2295    0]
 [   0  188 2300    0]
 [   4  228 2273    0]]


BONUS TASKS

ALMOST EQUAL WEIGHTS TO ALL EMBEDDINGS  [0.3, 0.3, 0.4]

In [ ]:
class ELMo_1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo_1, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        self.alphas = [0.3, 0.3, 0.4]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        # print("lstm1 : ", lstm_out1.shape)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)

        embed_out = embedded_x*self.alphas[0] + lstm_out1*self.alphas[1] + lstm_out2*self.alphas[2]
        return final_out, embed_out

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
embedding_dim = 100
hidden_dim = 50
num_layers = 2
batch_size_lstm = 32
vocab_size = len(vocabulary)
embedding_matrix = embedding_matrix.to(device)
model_1 = ELMo_1(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_1.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-28-a87145ec75f7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [ ]:
num_epochs = 2
path = "/content/gdrive/My Drive/ass2_model_1.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        outputs, _ = model_1(torch.tensor(batch_input_embeddings))
        outputs = outputs.float()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model_1.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-118-2b9d0472d624>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model_1(torch.tensor(batch_input_embeddings))
 50%|█████     | 1/2 [17:12<17:12, 1032.28s/it]

Epoch [1/2], Loss: 1.3669010701520103


100%|██████████| 2/2 [34:21<00:00, 1030.91s/it]

Epoch [2/2], Loss: 0.7110557462942032


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class downstream_task(nn.Module):
    def __init__(self, embedding_dim, vocab_size, no_of_values, num_layers=2):
        super(downstream_task, self).__init__()
        self.vocab_size = vocab_size
        self.no_of_values = no_of_values
        # self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers, batch_first=True).double()
        self.linear = nn.Linear(embedding_dim, no_of_values).double()

    def forward(self, input):
        # lstm_out, (hidden, cell_state) = self.lstm(input)
        # print("hidden shape", hidden.shape)
        # linear_out = self.linear(hidden[1])
        # print("linear_out shape", linear_out.shape)

        linear_out = self.linear(input)
        return linear_out

In [ ]:
model_downstream = downstream_task(embedding_dim, vocab_size, 4).to(device)

# num_params = sum(p.numel() for p in model_downstream.parameters() if p.requires_grad)
# print(f"Number of trainable parameters: {num_params}")

no_of_values = 4

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_downstream.parameters(), lr=learning_rate, weight_decay=1e-5)
# train_val = [int(val)-1 for val in train_value]

In [ ]:
import torch

def custom_collate_fn(batch):
    max_length = max(len(sentence) for sentence, _ in batch)
    padded_inputs = []
    outputs = []
    for sentence, label in batch:
        # print(sentence)
        sos_token = [word2idx['<SOS>']]
        sentence_tokens = [word2idx[word] if word in word2idx else word2idx['<UNK>'] for word in sentence]
        pad_tokens = [word2idx['<PAD>']] * (max_length - len(sentence))
        combined_list = sos_token + sentence_tokens + pad_tokens

        padded_inputs.append(combined_list)
        outputs.append(label)


    # print(torch.tensor(padded_inputs).shape, torch.tensor(outputs).shape)
    return torch.tensor(padded_inputs), torch.tensor(outputs)


from torch.utils.data import DataLoader, Dataset
batch_size = 32

class CustomDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return [self.sentences[idx], self.labels[idx]]

batch_size = 1
custom_dataset = CustomDataset(train_sentence, train_val)
dataloader_downstream = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)

for batch_input_embeddings, batch_target in (dataloader_downstream):
    print(batch_input_embeddings, batch_target)
    print(batch_input_embeddings.shape, batch_target.shape)
    break


tensor([[37362, 51159, 14965, 14884, 33668, 47537,  2186, 20506,  1736, 14092,
         36486,  8926, 38021]]) tensor([2])
torch.Size([1, 13]) torch.Size([1])


In [ ]:
num_epochs = 2
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model_1(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_model_1.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-35-20d52bce0e03>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
 50%|█████     | 1/2 [05:44<05:44, 344.50s/it]

Epoch [1/2], Loss: 1.3830000228229122


100%|██████████| 2/2 [11:25<00:00, 342.92s/it]

Epoch [2/2], Loss: 1.374326054735841


In [ ]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_dev):

for batch_input_embeddings, batch_target in dataloader_downstream:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model_1(batch_input_embeddings)
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]

    outputs_downstream = model_1(outputs_sum).cpu().detach().numpy()

    # print("Input 1 : ", batch_input_embeddings.shape)
    # print("Output 1 : ", outputs.shape)
    # print(batch_input_embeddings)
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()

    true.append(batch_target.cpu())
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    if(total == 10000):
        break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.26
0.13
0.08666666666666667
0.065
0.1
0.13666666666666666
0.16857142857142857
0.17625
0.18
0.189
0.1918181818181818
0.19833333333333333
0.20692307692307693
0.21142857142857144
0.218
0.22
0.22588235294117648
0.2338888888888889
0.2363157894736842
0.235
0.23761904761904762
0.24636363636363637
0.2508695652173913
0.25583333333333336
0.2656
0.27115384615384613
0.2722222222222222
0.27214285714285713
0.27379310344827584
0.2786666666666667
0.2764516129032258
0.2775
0.2772727272727273
0.27852941176470586
0.2825714285714286
0.2841666666666667
0.28675675675675677
0.2889473684210526
0.2917948717948718
0.29425
0.2897560975609756
0.29
0.28930232558139535
0.28909090909090907
0.29044444444444445
0.29043478260869565
0.29085106382978726
0.2916666666666667
0.293265306122449
0.296
0.2952941176470588
0.2948076923076923
0.2958490566037736
0.29555555555555557
0.29672727272727273
0.2982142857142857
0.29736842105263156
0.2977586206896552
0.2994915254237288
0.3006666666666667
0.3026229508196721
0.3048387096774

In [ ]:
true = [x.item() for x in true]
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.3193
Recall: 0.3193
F1 Score: 0.3193
Confusion Matrix:
[[ 764 1093  656   10]
 [ 544 1337  455    2]
 [ 538  858 1080    1]
 [ 656 1212  782   12]]


MORE WEIGHTAGE TO LAST LAYER

In [ ]:
class ELMo_2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo_2, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        self.alphas = [0.1, 0.3, 0.6]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        # print("lstm1 : ", lstm_out1.shape)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)

        embed_out = embedded_x*self.alphas[0] + lstm_out1*self.alphas[1] + lstm_out2*self.alphas[2]
        return final_out, embed_out

In [ ]:
model_2 = ELMo_2(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_2.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-46-55fa4cc7ecca>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [ ]:
num_epochs = 2
path = "/content/gdrive/My Drive/ass2_model_2.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        outputs, _ = model_2(torch.tensor(batch_input_embeddings))
        outputs = outputs.float()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model_2.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-48-27c5e191c726>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model_2(torch.tensor(batch_input_embeddings))
 50%|█████     | 1/2 [16:51<16:51, 1011.07s/it]

Epoch [1/2], Loss: 1.411609350664275


100%|██████████| 2/2 [33:41<00:00, 1010.99s/it]

Epoch [2/2], Loss: 0.7076889732508432


In [ ]:
num_epochs = 2
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model_2(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_model_2.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-49-70cd929a22ac>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
 50%|█████     | 1/2 [19:13<19:13, 1153.37s/it]

Epoch [1/2], Loss: 0.5411043438941209


100%|██████████| 2/2 [38:29<00:00, 1154.85s/it]

Epoch [2/2], Loss: 1.0836902089036586


In [ ]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_dev):

for batch_input_embeddings, batch_target in dataloader_downstream:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model_2(batch_input_embeddings)
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]

    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    outputs_downstream = model_downstream(outputs_sum).cpu().detach().numpy()

    true.append(batch_target.cpu())
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    if(total == 1000):
        break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.43
0.37
0.3466666666666667
0.355
0.368
0.39666666666666667
0.42428571428571427
0.43875
0.44666666666666666
accuracy :  0.444


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
true = [x.item() for x in true]
# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.444
Recall: 0.444
F1 Score: 0.444
Confusion Matrix:
[[124  27  40  21]
 [ 37  65  28  12]
 [ 56  10  86  22]
 [149  47 107 169]]


MODEL WITH MORE WEIGHTAGE TO EMBEDDING INPUTS





In [ ]:
class ELMo_3(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, weights_matrix):
        super(ELMo_3, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, batch_first=True).double()
        self.output_layer = nn.Linear(embedding_dim, vocab_size).double()
        self.alphas = [0.7, 0.2, 0.1]

    def forward(self, x):
        embedded_x = self.embedding_layer(x)
        lstm_out1, _ = self.lstm1(embedded_x)
        # print("lstm1 : ", lstm_out1.shape)
        lstm_out2, _ = self.lstm2(lstm_out1)
        # print("lstm2 : ", lstm_out2.shape)
        final_out = self.output_layer(lstm_out2)

        embed_out = embedded_x*self.alphas[0] + lstm_out1*self.alphas[1] + lstm_out2*self.alphas[2]
        return final_out, embed_out

In [ ]:
model_3 = ELMo_3(vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model_3.parameters(), lr=0.01, weight_decay=1e-5)

<ipython-input-56-ac39aaaa063e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(weights_matrix), freeze=False)


In [ ]:
num_epochs = 2
path = "/content/gdrive/My Drive/ass2_model_3.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        outputs, _ = model_3(torch.tensor(batch_input_embeddings))
        outputs = outputs.detach()
        outputs = outputs.float()
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model_3.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-58-4aa55795b778>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs, _ = model_3(torch.tensor(batch_input_embeddings))
 50%|█████     | 1/2 [16:50<16:50, 1010.86s/it]

Epoch [1/2], Loss: 1.4562356450784775


100%|██████████| 2/2 [33:41<00:00, 1010.83s/it]

Epoch [2/2], Loss: 0.7252356590486708


In [ ]:
num_epochs = 2
prev_loss = 1000
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count = 0
    mini = 100
    for batch_input_embeddings, batch_target in (dataloader_downstream):
        loss = 0
        batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)
        batch_target = batch_target.to(device)
        optimizer.zero_grad()
        _, outputs = model_3(batch_input_embeddings)
        # print("outputs",outputs.shape)
        outputs = outputs.detach()
        outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
        # print("outputs_downstream",outputs_downstream.shape)
        outputs_downstream = model_downstream(outputs_sum)
        # print("outputs_downstream",outputs_downstream.shape)
        loss = loss_function(outputs_downstream, batch_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if prev_loss > total_loss:
        torch.save(model_downstream.state_dict(), "/content/gdrive/My Drive/ass2_downstream_model_3.pth")
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader_downstream)}')


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-59-90ee4cde166c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_embeddings = torch.tensor(batch_input_embeddings).to(device)


In [ ]:
count = 0
total = 0

true = []
predicted = []
# for batch_input_embeddings, batch_target in tqdm(dataloader_downstream_dev):


for batch_input_embeddings, batch_target in dataloader_downstream:
    batch_input_embeddings = batch_input_embeddings.to(device)
    batch_target = batch_target.to(device)
    _, outputs = model_2(batch_input_embeddings)
    outputs = outputs.detach()
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]

    outputs_downstream = model_downstream(outputs_sum).cpu().numpy()

    # print("Input 1 : ", batch_input_embeddings.shape)
    # print("Output 1 : ", outputs.shape)
    # print(batch_input_embeddings)
    outputs_down = np.argmax((torch.tensor(outputs_downstream)).cpu().numpy())
    outputs_sum = torch.sum(outputs, dim=1)/outputs.shape[1]
    # outputs_sum = outputs_sum.cpu().numpy()
    # print("argmax : ", outputs_down)

    outputs_downstream = model_downstream(outputs_sum).cpu().numpy()
    # print("Output 2 : ", outputs_downstream.shape)



    true.append(batch_target.cpu())  # Move the true labels to CPU
    predicted.append(outputs_down)
    if outputs_down == batch_target:
        count += 1
    total += 1
    if(total == 1000):
        break
    if(total%100 == 0):
        print(count / total)

print("accuracy : ", count / total)


0.43
0.37
0.3466666666666667
0.355
0.368
0.39666666666666667
0.42428571428571427
0.43875
0.44666666666666666
accuracy :  0.444


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
true = [x.item() for x in true]
# Calculate precision, recall, and F1 score with 'micro' average
precision = precision_score(true, predicted, average='micro')
recall = recall_score(true, predicted, average='micro')
f1 = f1_score(true, predicted, average='micro')
conf_matrix = confusion_matrix(true, predicted)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.444
Recall: 0.444
F1 Score: 0.444
Confusion Matrix:
[[124  27  40  21]
 [ 37  65  28  12]
 [ 56  10  86  22]
 [149  47 107 169]]


character convolutional network in ELM0

In [39]:
char_train = train_sentence
char_label = train_value


In [40]:
char_vocabulary = set()
char2idx = {}
idx2char = {}

for sentence in char_train:
    for word in sentence:
        for char in word:
            char_vocabulary.add(char)

In [49]:
char_to_index = {char: idx for idx, char in enumerate(char_vocabulary)}
char_vocab_size = len(char_vocabulary)
def get_one_hot_encode(char, char_vocabulary_size):
    one_hot = torch.zeros(char_vocabulary_size)
    one_hot[char_to_index.get(char, 0)] = 1
    return one_hot

char_vector = get_one_hot_encode('b', len(char_vocabulary))
print(char_vector)

char_embedding_dim = 10
char_hidden_dim = 20

char_embedding_layer = nn.Embedding(len(char_vocabulary), char_embedding_dim)

char_embedding = char_embedding_layer(torch.LongTensor([char_to_index['b']]))
print(char_embedding)

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[-0.6291, -0.3937, -1.0740, -0.2474, -0.9265,  1.3926, -0.6506,  0.5153,
         -0.2007, -0.2152]], grad_fn=<EmbeddingBackward0>)


In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CharCNN(nn.Module):
    def __init__(self, vocab_size, char_embedding_dim, char_hidden_dim):
        super(CharCNN, self).__init__()
        self.char_embedding = nn.Embedding(vocab_size, char_embedding_dim)
        self.char_conv1 = nn.Conv1d(char_embedding_dim, char_hidden_dim, kernel_size=3)
        self.char_conv2 = nn.Conv1d(char_embedding_dim, char_hidden_dim, kernel_size=4)
        self.char_conv3 = nn.Conv1d(char_embedding_dim, char_hidden_dim, kernel_size=5)

    def forward(self, x):
        char_embedded = self.char_embedding(x)
        char_embedded = char_embedded.permute(0, 2, 1)  # Swap dimensions for Conv1d
        char_out1 = F.relu(self.char_conv1(char_embedded))
        char_out2 = F.relu(self.char_conv2(char_embedded))
        char_out3 = F.relu(self.char_conv3(char_embedded))
        char_out1 = F.max_pool1d(char_out1, char_out1.size(2)).squeeze(2)
        char_out2 = F.max_pool1d(char_out2, char_out2.size(2)).squeeze(2)
        char_out3 = F.max_pool1d(char_out3, char_out3.size(2)).squeeze(2)
        char_out = torch.cat([char_out1, char_out2, char_out3], dim=1)
        return char_out

class ELMoChar(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, char_embedding_dim, char_hidden_dim, hidden_dim):
        super(ELMoChar, self).__init__()
        self.embedding_dim = char_hidden_dim * 3  # Concatenate character CNN output
        self.hidden_dim = hidden_dim
        self.char_cnn = CharCNN(char_vocab_size, char_embedding_dim, char_hidden_dim)
        self.lstm1 = nn.LSTM(input_size=self.embedding_dim, hidden_size=hidden_dim, dropout=0.3, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=self.embedding_dim, hidden_size=hidden_dim, dropout=0.3, bidirectional=True)
        self.output_layer = nn.Linear(self.embedding_dim, vocab_size)

    def forward(self, char_inputs):
        char_cnn_out = self.char_cnn(char_inputs)
        lstm_out1, _ = self.lstm1(char_cnn_out.unsqueeze(1))  # Add an additional dimension
        lstm_out2, _ = self.lstm2(lstm_out1)
        final_out = self.output_layer(lstm_out2)
        return final_out


In [51]:
model_ELMoChar = ELMoChar(vocab_size, char_vocab_size, char_embedding_dim, char_hidden_dim, hidden_dim).to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
# learning_rate = 0.001
optimizer = optim.Adam(model_ELMoChar.parameters(), lr=0.01, weight_decay=1e-5)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
num_epochs = 5
path = "/content/gdrive/My Drive/ass2_model_dropout_char.pth"
prev_loss = 1000
from tqdm import tqdm

for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    count=0
    mini = 100
    for batch_input_embeddings, batch_target_embeddings in (dataloader):
        loss = 0
        batch_input_embeddings = batch_input_embeddings.to(device)
        batch_target_embeddings = batch_target_embeddings.to(device)

        # print(batch_input_embeddings.shape, batch_target_embeddings.shape)
        optimizer.zero_grad()
        outputs, _ = model_ELMoChar(torch.tensor(batch_input_embeddings))
        # print(batch_target_embeddings.shape, outputs.shape)
        loss = loss_function(outputs.view(-1,vocab_size),batch_target_embeddings.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if(prev_loss > total_loss):
        torch.save(model.state_dict(), path)
    prev_loss = total_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader)}')
